In [1]:
import os
from langchain.llms import HuggingFaceTextGenInference
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import langchain
langchain.__version__

'0.1.16'

Wir definieren nun zwei Hilfsfunktionen um einfach zwischen GPT4 und unserem lokalen Sprachmodell zu wechseln.

In [3]:
def getLLM_openai(): #openai API key kommt aus der .env Datei
    return ChatOpenAI(model="gpt-4-turbo-preview")
    #return OpenAI(request_timeout=10, model_name="gpt-4-turbo-preview") #10s timeout, model_name = "gpt-3.5-turbo"


def getLLM_TGI(): 
     llm = HuggingFaceTextGenInference(inference_server_url=os.environ['TGI_URL'], max_new_tokens=2048, top_k=10, top_p=0.95, typical_p=0.95, temperature=0.01, repetition_penalty=1.03) 
     return llm
 
 
llm = getLLM_openai()

/opt/anaconda3/envs/ml/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Erste Experimente
In der nächsten Zelle rufen wir das LLM einfach auf.

In [4]:
res = llm.invoke("Warum ist der Himmel blau? Antworte in drei Sätzen.")
print(res)

content='Der Himmel erscheint blau, weil die Moleküle in der Erdatmosphäre das blaue Licht der Sonne stärker streuen als das Licht anderer Farben. Dieses Phänomen wird als Rayleigh-Streuung bezeichnet. Da kurzwelliges, blaues Licht in alle Richtungen zerstreut wird, sehen wir den Himmel über uns in dieser Farbe.' response_metadata={'token_usage': <OpenAIObject at 0x106206f40> JSON: {
  "prompt_tokens": 25,
  "completion_tokens": 88,
  "total_tokens": 113
}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_daebbecb8f', 'finish_reason': 'stop', 'logprobs': None} id='run-46e78afc-e913-4aaa-ad83-38fe420b4f92-0'


# RAG workflow
Im folgenden stellen wir einen Beispielhaften RAG Workflow dar, der Daten von der Wikipedia Seite von Tibet als Grundlage nimmt.

In [5]:
import prompts
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, GPT4AllEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = WebBaseLoader("https://de.wikipedia.org/wiki/Tibet")
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
splits = text_splitter.split_documents(loader.load())

Wir definieren uns nun ein Embedding für die deutsche Sprache und nehmen Chroma als Vektordatenbank. Diese benötigen wir für die Suche nach passenden Dokumenten für unsere Anfrage an das Sprachmodell

In [8]:
#emb = HuggingFaceEmbeddings(model_name='aari1995/German_Semantic_STS_V2')
emb = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents=splits, embedding=emb) # , persist_directory="./chroma_db"
retriever = vectorstore.as_retriever()
rag_prompt = prompts.german_sauerkrautlm()

In der folgenden Zelle verketten wir die Anfrage an die Vektordatenbank mit dem Aufruf von unserem Sprachmodell in einem `rag_chain`.

In [9]:
# configure rag chain
chain_type_kwargs = {"prompt": rag_prompt}
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, 
                                    chain_type_kwargs=chain_type_kwargs,
                                    return_source_documents=True)

Nun erfolgt die eigentliche Anfrage an das LLM:

In [10]:
res = rag_chain.invoke("Was weißt du zu Tibet?")
print(res['result'])

Tibet ist eine Region in Zentralasien, die eine eigenständige Kultur und Staaten wie Shangshung und Tubo vor dem 7. Jahrhundert entwickelt hat. Es ist geografisch durch das Himalaya-Gebirge abgeschieden und erstreckt sich über das tibetische Hochland. Tibet geriet Mitte des 13. Jahrhunderts unter mongolische Herrschaft, zusammen mit China. Bis ins 20. Jahrhundert hatte Tibet ein eigenes Staatswesen, doch seine gegenwärtige Zugehörigkeit zur Volksrepublik China ist völkerrechtlich umstritten. Seit 1959 existiert eine tibetische Exilregierung, die international Unterstützung findet, jedoch offiziell nicht anerkannt ist.
